In [ ]:
%pip install langchain langchain-openai langchain-community python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# --- 1. Environment and API Key Setup ---
# Load environment variables from .env file
load_dotenv()

# --- 2. Session-Based History Store ---
# This dictionary will store chat histories for different sessions
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Retrieves a chat message history for a given session ID.
    If the session ID does not exist, a new history is created.
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# --- 3. The Main Chat Function (Corrected) ---

# Initialize the Chat Model (can be done once)
llm = ChatOpenAI(model="gpt-4o")

# Create the prompt template with a placeholder for the AI's role
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "{ai_role}"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}"),
])

# Create the primary chain
chain = prompt_template | llm

# Create the final chain with history management
# This is the key change: using RunnableWithMessageHistory
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

def run_contextual_chat(session_id: str, user_input: str, ai_role: str):
    """
    Runs a contextual chat session using the corrected chain.

    Args:
        session_id: A unique identifier for the conversation (e.g., "intel123").
        user_input: The message from the user.
        ai_role: The system role for the AI (e.g., "You are a master programmer.").
    """
    # Define the configuration for this specific run, including the session_id
    config = {"configurable": {"session_id": session_id}}

    # Invoke the chain with the necessary inputs and configuration
    response = chain_with_history.invoke(
        {"input": user_input, "ai_role": ai_role},
        config=config,
    )

    return response.content

# --- 4. Example Usage (No changes needed here) ---

# Define a session ID and the AI's role
my_session_id = "intel123"
my_ai_role = "You are a sarcastic assistant who is secretly a pirate."

# First interaction
print("--- First Interaction ---")
response1 = run_contextual_chat(my_session_id, "My name is Alex. What's the weather like in Tortuga?", my_ai_role)
print(f"AI: {response1}\n")

# Second interaction in the same session
print("--- Second Interaction (same session) ---")
response2 = run_contextual_chat(my_session_id, "Do you remember my name?", my_ai_role)
print(f"AI: {response2}\n")

# Start a new, separate conversation with a different session ID and role
print("--- New Conversation (different session) ---")
new_session_id = "nasa456"
new_ai_role = "You are a helpful NASA scientist explaining complex topics simply."
response3 = run_contextual_chat(new_session_id, "What is a black hole?", new_ai_role)
print(f"AI: {response3}")
